# Segmenting and Clustering Neighborhoods in Toronto - part 2

## retrieve and clean dataframe from wikipedia web page

In [1]:
import pandas as pd
import numpy as np
import urllib.request
!conda install -c conda-forge beautifulsoup4 --yes
print('finished!')
import bs4 as bs

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source)

table = soup.find('table')
table_rows = table.find_all('tr')

wiki_table = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    wiki_table.append(row)
    
df = pd.DataFrame(wiki_table, columns=["PostalCode", "Borough", "Neighborhood"])
df = df.iloc[1:]
df.replace(r'\s', '', regex = True, inplace = True)
df = df[df.Borough != "Notassigned"]
df = df.reset_index(drop=True)

index = 0
size = len(df)-1
while index < size:
    if df.iloc[index]['PostalCode'] == df.iloc[index+1]["PostalCode"]:
        df.iloc[index]['Neighborhood'] += ", "+df.iloc[index+1]['Neighborhood']
        df.drop(index+1, inplace=True)
        df = df.reset_index(drop=True)
        index -= 1
        size -= 1
    index += 1
    
index = 0
size = len(df)-1

while index < size:
    if df.iloc[index]['Neighborhood'] == 'Notassigned':
        df.iloc[index]['Neighborhood'] = df.iloc[index]['Borough']
    index += 1

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-2.11                  |         openblas          10 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-for

In [2]:
df.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,NorthYork,Parkwoods
1,M4A,NorthYork,VictoriaVillage
2,M5A,DowntownToronto,Harbourfront
3,M6A,NorthYork,"LawrenceHeights, LawrenceManor"
4,M7A,Queen'sPark,Queen'sPark
5,M9A,Queen'sPark,Queen'sPark
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,NorthYork,DonMillsNorth
8,M4B,EastYork,"WoodbineGardens, ParkviewHill"
9,M5B,DowntownToronto,"Ryerson, GardenDistrict"


## retrieve latitudes and longitudes and add them to the df

In [10]:
postal_df = pd.read_csv('http://cocl.us/Geospatial_data', delimiter=',')
postal_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
new_df =  df.merge(postal_df, left_on='PostalCode', right_on='Postal Code')
new_df.drop('Postal Code', axis=1, inplace=True)
new_df.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,VictoriaVillage,43.725882,-79.315572
2,M5A,DowntownToronto,Harbourfront,43.654260,-79.360636
3,M6A,NorthYork,"LawrenceHeights, LawrenceManor",43.718518,-79.464763
4,M7A,Queen'sPark,Queen'sPark,43.662301,-79.389494
5,M9A,Queen'sPark,Queen'sPark,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,NorthYork,DonMillsNorth,43.745906,-79.352188
8,M4B,EastYork,"WoodbineGardens, ParkviewHill",43.706397,-79.309937
9,M5B,DowntownToronto,"Ryerson, GardenDistrict",43.657162,-79.378937
